In [4]:
from ortools.linear_solver import pywraplp

In [15]:
def create_data_model():
    """Ví dụ này sẽ tạo 1 the data"""
    data = {}
    weights = [48, 30, 19, 36, 36, 27, 42, 42, 36, 24, 30]
    data['weights'] = weights
    data['items'] = list(range(len(weights)))
    data['bins'] = data['items']
    data['bin_capacity'] = 100
    return data

In [17]:
# Tao the mip solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver('SCIP')
data = create_data_model()

In [18]:
# Variables
# x[i,j] = 0 nếu item i được cất ở trên bin j.
x = {}
for i in data['items']:
    for j in data['bins']:
        x[(i,j)] = solver.IntVar(0, 1, 'x_%i_%i' % (i, j))
#y[j] = 1 nếu bin j được sử dụng
y = {}
for j in data['bins']:
    y[j] = solver.IntVar(0, 1, 'y[%i]' % j)

In [21]:
# Rằng buộc:
# Mỗi items chỉ được đặt, phải đặt đúng 1 bin.
for i in data['items']:
    solver.Add(sum(x[i,j] for j in data['bins']) == 1)
# Số lượng đồ được đặt ở mỗi bin ko vượt quá trọng tải của bin đó
for j in data['bins']:
    solver.Add(
        sum(x[i,j]*data['weights'][i] for i in data['items']) <= y[j]*data["bin_capacity"])

In [33]:
# objective minimize the number of bins used
solver.Minimize(solver.Sum(y[j] for j in data['bins']))

In [34]:
status = solver.Solve()
if status == pywraplp.Solver.OPTIMAL:
    num_bins = 0.
    for j in data['bins']:
        if y[j].solution_value() == 1:
            bin_items = []
            bin_weight = 0
            for i in data['items']:
                if x[i, j].solution_value() > 0:
                    bin_items.append(i)
                    bin_weight += data['weights'][i]
            if bin_weight > 0:
                num_bins += 1
                print('bin so may', j)
                print(' nhung item nao packed', bin_items)
                print(' Total weight:' , bin_weight)
                print()
    print()
    print('so luong bin su dung:', num_bins)
    print('Thoi = ', solver.WallTime(),'miliseconds')
else:
    print('Bai toan ko co nghiem toi uu')


bin so may 0
 nhung item nao packed [0, 1, 2]
 Total weight: 97

bin so may 1
 nhung item nao packed [3, 4, 5]
 Total weight: 99

bin so may 2
 nhung item nao packed [6, 7]
 Total weight: 84

bin so may 3
 nhung item nao packed [8, 9, 10]
 Total weight: 90


so luong bin su dung: 4.0
Thoi =  1008545 miliseconds
